In [34]:
import pandas as pd
import numpy as np
import matplotlib as mplt
import keras 

from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping

In [35]:
test_data = pd.read_csv("test.csv")
train_data = pd.read_csv("train.csv")

In [36]:
x = train_data.iloc[:,1:]
y = train_data.iloc[:,0]

In [37]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test =  train_test_split(x,y,test_size=0.1)

In [38]:
x_train = x_train.values.reshape(-1,28,28,1)
x_test = x_test.values.reshape(-1,28,28,1)

In [39]:

datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
            zoom_range = 0.1, # Randomly zoom image 
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=False,  # randomly flip images
            vertical_flip=False)  # randomly flip images

In [40]:
x_train = x_train.astype("float32")/255
x_test = x_test.astype("float32")/255
datagen.fit(x_train)

In [41]:
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

In [42]:

model = Sequential()
model.add(Conv2D(filters = 32,activation='relu',strides=1,padding = 'same', kernel_size=(3,3),data_format='channels_last',input_shape =(28,28,1)))
model.add(BatchNormalization())
model.add(Conv2D(filters = 32,activation='relu',strides=1,padding='same',kernel_size=(3,3),data_format='channels_last'))
model.add(BatchNormalization())
model.add(MaxPooling2D(strides=2,pool_size=(2,2),padding='valid'))
model.add(Dropout(0.25))          

model.add(Conv2D(filters = 64,activation = 'relu',strides=1,padding='same',kernel_size=(3,3),data_format='channels_last'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 64,activation = 'relu',strides=1,padding='same',kernel_size=(3,3),data_format='channels_last'))
model.add(BatchNormalization())
model.add(MaxPooling2D(strides=2,pool_size=(2,2),padding='valid'))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(1024,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(10,activation='softmax'))

In [43]:
optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999 )

In [44]:
model.compile(optimizer=optimizer,loss="categorical_crossentropy",metrics=["accuracy"])

In [45]:
reduce_lr = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)

In [46]:
batch_size = 64
epochs = 50
history = model.fit(datagen.flow(x_train,y_train,batch_size=batch_size),epochs=epochs,
validation_data=(x_test,y_test),verbose=1,steps_per_epoch=x_train.shape[0]//batch_size,callbacks=[reduce_lr])

Epoch 1/50
590/590 [==============================] - 32s 50ms/step - loss: 0.3787 - accuracy: 0.8882 - val_loss: 0.0810 - val_accuracy: 0.9743 - lr: 0.0010
Epoch 2/50
590/590 [==============================] - 29s 50ms/step - loss: 0.1295 - accuracy: 0.9605 - val_loss: 0.0471 - val_accuracy: 0.9871 - lr: 9.0000e-04
Epoch 3/50
590/590 [==============================] - 30s 50ms/step - loss: 0.0977 - accuracy: 0.9710 - val_loss: 0.0377 - val_accuracy: 0.9900 - lr: 8.1000e-04
Epoch 4/50
590/590 [==============================] - 33s 56ms/step - loss: 0.0752 - accuracy: 0.9777 - val_loss: 0.0374 - val_accuracy: 0.9895 - lr: 7.2900e-04
Epoch 5/50
590/590 [==============================] - 30s 50ms/step - loss: 0.0690 - accuracy: 0.9780 - val_loss: 0.0294 - val_accuracy: 0.9905 - lr: 6.5610e-04
Epoch 6/50
590/590 [==============================] - 30s 50ms/step - loss: 0.0606 - accuracy: 0.9817 - val_loss: 0.0413 - val_accuracy: 0.9883 - lr: 5.9049e-04
Epoch 7/50
590/590 [==================